**1. Download/Load SP500 stocks prices data.**

In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
pip install statsmodels

In [ ]:
pip install pandas_datareader

In [ ]:
pip install yfinance

In [ ]:
pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.8/247.8 kB 8.7 MB/s eta 0:00:00


In [ ]:
pip install sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
pip install PyPortfolioOpt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.2 MB/s eta 0:00:00


In [ ]:
pip install pandas-ta

In [ ]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-08-02'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()
df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%%**********************]  503 of 503 completed


Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-08-04 A        43.978603   46.830002   47.330002   46.740002   47.160000   
           AAL      32.367760   33.580002   33.770000   33.000000   33.570000   
           AAPL     24.441740   26.467501   26.500000   26.320000   26.395000   
           ABBV     46.825123   66.099998   66.980003   66.080002   66.580002   
           ABT      38.708569   44.810001   45.090000   44.779999   45.060001   
...                       ...         ...         ...         ...         ...   
2024-08-01 XYL     133.330002  133.330002  134.740005  131.679993  133.020004   
           YUM     134.500000  134.500000  134.750000  131.660004  133.460007   
           ZBH     111.120003  111.120003  113.419998  110.260002  111.839996   
           ZBRA    342.579987  342.579987  353.859985  334.299988  350.420013   
           ZTS     182.419998  182.419998  183.929993  180.600006  181.050003   

Price                   volume  
date       ticker               
2016-08-04 A         1700200.0  
           AAL       6384700.0  
           AAPL    109634800.0  
           ABBV      4901900.0  
           ABT       6583500.0  
...                        ...  
2024-08-01 XYL       1591700.0  
           YUM       1627800.0  
           ZBH       1129900.0  
           ZBRA       580900.0  
           ZTS       1987800.0  

[990341 rows x 6 columns]

2. Calculate features and technical indicators for each stock.



*   Garman-Klass Volatility
*   RSI
*   Bollinger Bands
*   ATR
*   MACD
*   Dollar Volume


\begin{equation}
\text{Garman-Klass Volatility} = \frac{(\ln(\text{High}) - \ln(\text{Low}))^2}{2} - (2\ln(2) - 1)(\ln(\text{Adj Close}) - \ln(\text{Open}))^2
\end{equation}









In [ ]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0] if pandas_ta.bbands(close=np.log1p(x), length=20) is not None else np.nan)

df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1] if pandas_ta.bbands(close=np.log1p(x), length=20) is not None else np.nan)

df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2] if pandas_ta.bbands(close=np.log1p(x), length=20) is not None else np.nan)

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20)
    if macd is not None:
        return macd.iloc[:,0]
    else:
        return pd.Series([np.nan] * len(close), index=close.index)

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df


Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-08-04 A        43.978603   46.830002   47.330002   46.740002   47.160000   
           AAL      32.367760   33.580002   33.770000   33.000000   33.570000   
           AAPL     24.441740   26.467501   26.500000   26.320000   26.395000   
           ABBV     46.825123   66.099998   66.980003   66.080002   66.580002   
           ABT      38.708569   44.810001   45.090000   44.779999   45.060001   
...                       ...         ...         ...         ...         ...   
2024-08-01 XYL     133.330002  133.330002  134.740005  131.679993  133.020004   
           YUM     134.500000  134.500000  134.750000  131.660004  133.460007   
           ZBH     111.120003  111.120003  113.419998  110.260002  111.839996   
           ZBRA    342.579987  342.579987  353.859985  334.299988  350.420013   
           ZTS     182.419998  182.419998  183.929993  180.600006  181.050003   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2016-08-04 A         1700200.0         -0.001806        NaN       NaN   
           AAL       6384700.0         -0.000248        NaN       NaN   
           AAPL    109634800.0         -0.002260        NaN       NaN   
           ABBV      4901900.0         -0.047768        NaN       NaN   
           ABT       6583500.0         -0.008893        NaN       NaN   
...                        ...               ...        ...       ...   
2024-08-01 XYL       1591700.0          0.000262  44.154676  4.889407   
           YUM       1627800.0          0.000246  56.324268  4.834474   
           ZBH       1129900.0          0.000383  51.618057  4.669410   
           ZBRA       580900.0          0.001419  61.511205  5.734028   
           ZTS       1987800.0          0.000145  59.298044  5.160969   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2016-08-04 A            NaN       NaN       NaN       NaN      74.772421  
           AAL          NaN       NaN       NaN       NaN     206.658435  
           AAPL         NaN       NaN       NaN       NaN    2679.665281  
           ABBV         NaN       NaN       NaN       NaN     229.532070  
           ABT          NaN       NaN       NaN       NaN     254.837861  
...                     ...       ...       ...       ...            ...  
2024-08-01 XYL     4.928586  4.967765  1.856144 -0.439766     212.221364  
           YUM     4.867867  4.901261  0.782916 -0.258230     218.939100  
           ZBH     4.704101  4.738791 -0.504137  0.447551     125.554491  
           ZBRA    5.795394  5.856761  0.683337  7.910073     199.004714  
           ZTS     5.192383  5.223798  0.710924  2.150326     362.614472  

[990341 rows x 14 columns]

3. Aggregate to monthly level and filter top 150 most liquid stocks for each month.



*   To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.

In [ ]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('M').last().stack('ticker')],
                  axis=1)).dropna()

data

dollar_volume   adj close  garman_klass_vol        rsi  \
date       ticker                                                           
2016-09-30 A           79.965589   44.332623         -0.000643  54.145119   
           AAL        272.390873   35.288372          0.000336  54.527521   
           AAPL      4730.544126   26.099354         -0.002041  58.107449   
           ABBV       296.328713   44.678680         -0.044150  40.471316   
           ABT        277.293747   36.531696         -0.006690  47.372179   
...                          ...         ...               ...        ...   
2024-08-31 XYL        212.221364  133.330002          0.000262  44.154676   
           YUM        218.939100  134.500000          0.000246  56.324268   
           ZBH        125.554491  111.120003          0.000383  51.618057   
           ZBRA       199.004714  342.579987          0.001419  61.511205   
           ZTS        362.614472  182.419998          0.000145  59.298044   

                     bb_low    bb_mid   bb_high       atr      macd  
date       ticker                                                    
2016-09-30 A       3.754324  3.791448  3.828571 -1.343778 -0.080118  
           AAL     3.517487  3.587081  3.656676  0.214052 -0.011525  
           AAPL    3.219242  3.281954  3.344666 -1.229247  0.295692  
           ABBV    3.812678  3.834058  3.855438 -1.524399 -0.380746  
           ABT     3.588636  3.612017  3.635397 -1.251093 -0.380191  
...                     ...       ...       ...       ...       ...  
2024-08-31 XYL     4.889407  4.928586  4.967765  1.856144 -0.439766  
           YUM     4.834474  4.867867  4.901261  0.782916 -0.258230  
           ZBH     4.669410  4.704101  4.738791 -0.504137  0.447551  
           ZBRA    5.734028  5.795394  5.856761  0.683337  7.910073  
           ZTS     5.160969  5.192383  5.223798  0.710924  2.150326  

[47279 rows x 9 columns]

In [ ]:
data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2017-08-31 AAL      43.511288         -0.000146  37.643882  3.741600   
           AAPL     38.529816         -0.001368  65.283088  3.622255   
           ABBV     55.412022         -0.032460  68.425640  3.944742   
           ABT      45.090454         -0.004445  62.800545  3.773206   
           ACN     117.201630         -0.004114  59.510322  4.746614   
...                       ...               ...        ...       ...   
2024-08-31 VRTX    505.779999          0.000280  67.095734  6.172529   
           VZ       40.849998          0.000237  53.773673  3.687520   
           WFC      56.880001          0.000536  42.796472  4.063563   
           WMT      69.790001          0.000109  57.374348  4.245900   
           XOM     116.949997          0.000298  53.945671  4.716745   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2017-08-31 AAL     3.839023  3.936447  0.379471 -1.527069  
           AAPL    3.650530  3.678805 -1.153259  0.655525  
           ABBV    3.983809  4.022875 -1.529533  0.540382  
           ABT     3.799478  3.825750 -1.354432  0.315576  
           ACN     4.759914  4.773213 -1.342323  0.504321  
...                     ...       ...       ...       ...  
2024-08-31 VRTX    6.199367  6.226205  1.550018  7.417854  
           VZ      3.729088  3.770656 -0.160774 -0.004974  
           WFC     4.099536  4.135510  0.972818 -0.040741  
           WMT     4.262603  4.279306  0.878955  0.477354  
           XOM     4.756385  4.796026  0.693806  1.000067  

[12665 rows x 8 columns]